In [1]:
import boto3
from langchain_aws import ChatBedrock, BedrockEmbeddings
from config import aws_region, bedrock_model, embedding_model, temperature, max_tokens, pinecone_api_key, pinecone_index_name, namespaces


/Users/rohan/Desktop/Work/RAG/testvenv/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [ ]:
def get_bedrock_client(service_name: str = "bedrock-runtime", region: str = None):
    """
    Get AWS Bedrock client
    
    Args:
        service_name: AWS service name
        region: AWS region
        
    Returns:
        Boto3 client
    """
    if region is None:
        region = aws_region
    
    return boto3.client(service_name, region_name=region)

In [ ]:
def get_bedrock_embeddings(embedding_model: str, region: str = None) -> BedrockEmbeddings:
    """
    Get Bedrock embeddings model
    
    Args:
        embedding_model: Embedding model name available in AWS
        region: AWS region
        
    Returns:
        BedrockEmbeddings instance
    """
    client = get_bedrock_client("bedrock-runtime", region)
    return BedrockEmbeddings(
        client=client,
        model_id = embedding_model
    )


In [ ]:
def setup_bedrock_llm(config: Config) -> ChatBedrock:
    """
    Setup Bedrock LLM with configuration
    
    Args:
        config: Application configuration
        
    Returns:
        ChatBedrock instance
    """
    client = get_bedrock_client("bedrock-runtime", config.aws_region)
    
    # Base kwargs
    kwargs = {
        "client": client,
        "model_id": config.bedrock_model_id,
        "temperature": config.temperature,
        "max_tokens": config.max_tokens,
        "system_prompt": config.system_prompt
    }
    
    # Add guardrails if configured
    model_kwargs = config.get_model_kwargs()
    if model_kwargs:
        kwargs["model_kwargs"] = model_kwargs
    
    return ChatBedrock(**kwargs)

In [ ]:
# Index creation utils